In [7]:
from dotenv import load_dotenv
from os.path import join, dirname
from google.cloud import bigquery
import json 

In [8]:
dotenv_path = '.env'
load_dotenv(dotenv_path)
srcJsonPath = "/home/pardhu448/Projects/indusAction/redesignBocwGrievanceSystem/BoCWDataPipeline/BoCWDataPipeline/data/dailyMongoDump"

In [13]:
from datetime import datetime
import pytz 
tz = pytz.timezone("Asia/Kolkata")

def replaceDateTimeWithTZ(newDict, k, v):
        #from IPython.core.debugger import set_trace
        #set_trace()
        
        if k == 'created_at':    
            formatIST = '%d/%m/%Y %I:%M:%S %p'
            
        elif k in ['identification_data_created', 
                   'identification_data_modified', 
                   #'claims_documentation_data_created', 
                   #'claims_documentation_data_modified', 
                   'registration_documentation_data_created', 
                   'registration_documentation_data_modified',
                   'registration_application_data_created', 
                   'registration_application_data_modified']:
            #from IPython.core.debugger import set_trace
            #set_trace()
            formatIST = '%m/%d/%Y, %I:%M:%S %p'
            
        elif k in ['claims_documentation_data_created','claims_documentation_data_modified']:
            #from IPython.core.debugger import set_trace
            #set_trace()
            formatIST = '%m/%d/%Y, %I:%M:%S %p'
            
        else:
            return newDict
        
        #newDict.pop(k, None)
        formatUTC = '%d/%m/%Y %H:%M:%S'
        formattedDt = datetime.strptime(v, formatIST)
        formattedDtString = datetime.strftime(tz.normalize(tz.localize(formattedDt)).astimezone(pytz.utc), formatUTC)
        newDict[k] = formattedDtString            
        return newDict

In [14]:
def getFormattedData(newDict, k, v):
    #from IPython.core.debugger import set_trace
    #set_trace()
    
    import string
    #newDict.pop(k, None)
    #Remove blank spaces in fields
    newKey = k.replace(' ', '_')
    #Remove dollars in fields
    newKey = newKey.replace('$', '_')
    #Remove square brackets in fields
    table = str.maketrans('[]', '__')
    newKeyFinal = newKey.translate(table) 
    newDict[newKeyFinal] = v
    #from IPython.core.debugger import set_trace
    #set_trace()

    newDict = replaceDateTimeWithTZ(newDict, newKeyFinal, v)
    return newDict    

In [15]:
def parseJsonFile(jsonDict):
        new = {}                                          
        for k, v in jsonDict.items():                            
            k = k.encode('raw-unicode-escape').split(b'\\u')[0].decode()
            if isinstance(v, dict):
                v = parseJsonFile(v)
            new = getFormattedData(new, k, v)    
            #new = replaceDollarsInKeys(new, k, v)
            #new = replaceSquareBracketsInFields(new, k, v)
            #new = replaceDateTimeWithTZ(new, k, v)
        return new
    #jsonDollarsReplaced = replaceAllDollarsInKeys(jsonDict)
    #jsonTZReplaced = replaceDateTimeWithTZ(jsonDollarsReplaced)
    #return jsonTZReplaced
    #return jsonDollarsReplaced

In [16]:
import json
import ast
cmsDataInit = "/home/pardhu448/Projects/indusAction/redesignBocwGrievanceSystem/BoCWDataPipeline/BoCWDataPipeline/mongoinitdev/cms_dev_data.json"
contents = []
for eaLine in open(cmsDataInit, 'r'):
   eaLine = eaLine.replace('.', '_') 
   dataRecord = json.loads(eaLine)
   #from IPython.core.debugger import set_trace
   #set_trace()
   dataRecord['data'] = json.loads(dataRecord['data'])
   contents.append(dataRecord)
parsedJsonArray = [parseJsonFile(eaDoc) for eaDoc in contents[6:15]]
print(parsedJsonArray)

[{'mobile': '6204030448', 'state': 'Delhi BoCW 2021', 'created_at': '03/08/2022 06:52:37', 'operation': 'CREATE', 'data': {'mobile': '6204030448', 'state': 'Delhi BoCW 2021', 'persistent_data_statusOfAwarenessAboutClaims': 'aware', 'persistent_data_sourceOfInformation': 'missedCallHelpline', 'persistent_data_nameOfWorker': 'Dev Krishna Kumar', 'persistent_data_age': 25, 'persistent_data_genderOfTheWorker': 'male', 'persistent_data_area': 'Narela', 'persistent_data_pincode': 110040, 'persistent_data_district': 'northWestDelhi', 'persistent_data_statusOfLabourCard': 'available', 'persistent_data_labourCardNumberIfAvailable': 51380000262751, 'persistent_data_dateOfRegistrationOrLastRenewal': 'Fri Mar 19 2021', 'claims_documentation_id': '61aaeebf0a8065001a08fe1c', 'claims_documentation_status': 'Call Again', 'claims_documentation_assigneeMobile': '9810207515', 'claims_documentation_assigneeName': 'Ananya Iyer', 'claims_documentation_data_created': '03/12/2021 22:59:51', 'claims_documentat